In [1]:
#  This file is part of AISelect.

#  AISelect is free software: you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation, either version 3 of the License, or
#  (at your option) any later version.

#  AISelect is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with AISelect.  If not, see <http://www.gnu.org/licenses/>.


mypath = "Max"

import h5py
import os
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
from openpyxl import Workbook
import datetime
today = datetime.date.today()

onlydirs = [f for f in listdir(mypath) if isdir(join(mypath, f))]

samplelist = {}

In [2]:
onlydirs

['Auswertung']

In [3]:
h5filelist = []
for root, subdirs, files in os.walk(mypath):
    h5files = [join(root,f) for f in files if f.endswith(".h5")]
    h5filelist += h5files

In [4]:
h5filelist

['Max\\Auswertung\\p1\\Layer 2-3\\AIS M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-1\\M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-1.h5',
 'Max\\Auswertung\\p1\\Layer 2-3\\AIS M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-10\\M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-10.h5',
 'Max\\Auswertung\\p1\\Layer 2-3\\AIS M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-15\\M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-15.h5',
 'Max\\Auswertung\\p1\\Layer 2-3\\AIS M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-19\\M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-19.h5',
 'Max\\Auswertung\\p1\\Layer 2-3\\AIS M1 development_p1_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-647_630x_2-3-1-4\\M1 de

In [5]:
wb = Workbook()
ws = wb.active
count = 0 
from collections import defaultdict
layerfivelist = defaultdict(dict)
layertwolist = defaultdict(dict)
otherlist = {} 
ws.append(["Filename","Median","Standardeviation"])
for h5file in h5filelist:
    if h5py.is_hdf5(h5file):
        lala = h5py.File(h5file,"r")
        
        try:
            rois = [key for key in lala["Data"].keys()]
            lengths = []
            for roi in rois:
                length = lala["Data/"+roi+"/Physical"].attrs["AISPhysicalLength"]
                # length = lala["Data/"+roi+"/Pixeldata"].attrs["AISend"] - lala["Data/"+roi+"/Pixeldata"].attrs[
                # "AISstart"]
                if length < 0.1:
                    lengths.append(length*1024)
                else:
                    lengths.append(length)
        
            lengths = np.array(lengths)
            try:
                seriesname = lala.attrs["seriesname"].split("_")[7].split("-")[-2] # should evaluate to the animal
            except:
                seriesname = "Unknown"
            try:
                layername = h5file.split("Layer ")[1].split("\\")[0]
            except:
                layername = h5file.split("layer ")[1].split("\\")[0]
                
            pname = h5file.split("p")[1].split("\\")[0]
            if layername == "5":
                layerfivelist[pname][seriesname] = np.append(layerfivelist[pname].get(seriesname,np.array([])),lengths)
            elif layername == "2-3": 
                layertwolist[pname][seriesname] = np.append(layertwolist[pname].get(seriesname,np.array([])),lengths)
            else:
                print("ERROR on",seriesname)
                
                
        except KeyError as e :
            print("File {0} doesnt hava data Object: {1}".format(h5file,e))
            pass
        
    else:
        print("File " + h5file + " doesn't conform with h5 standards")
        pass
    
    count = 1
    if count > 5: break

In [6]:
layerfivelist

defaultdict(dict,
            {'1': {'1': array([ 12.12451699,  10.41127003,  11.07021117,  13.17882282,
                      12.91524636,  10.01590534,   6.45762318,  14.10134041,
                      21.08611651,  10.2794818 ,   7.24835255,  11.59736408,
                      10.54305825,  20.03181068,   8.96159952,  11.72915231,
                       9.09338774,  14.89206978,   7.51192901,  11.20199939,
                      10.93842294,  13.70597573,  11.59736408,  10.54305825,
                      10.93842294,   9.62054066,  16.20995206,   8.30265837,
                      10.67484648,  10.93842294,  16.07816384,   9.3569642 ,
                       9.09338774,  13.44239927,  15.02385801,   8.96159952,
                      11.46557585,  11.99272876,  10.67484648,   6.98477609,
                      10.67484648,   9.75232888,   5.79868204,   9.09338774,
                       8.4344466 ,   5.79868204,   5.40331735,   7.90729369,
                       8.69802306,   5.27152913,

In [7]:
wb = Workbook()
ws = wb.active
from openpyxl.cell import get_column_letter

for p, value in layertwolist.items():
    ws = wb.create_sheet("P {0}".format(p))
    for index,(key,item) in enumerate(value.items()):
        letter = get_column_letter(index+1)
        ws[letter+"1"] = "Animal " + key
        for index, length in enumerate(item):
            ws[letter+str(index+2)] = length
            
wb.save("Layer Two List.xlsx")

In [8]:
wb = Workbook()
ws = wb.active
from openpyxl.cell import get_column_letter

for p, value in layerfivelist.items():
    ws = wb.create_sheet("P {0}".format(p))
    for index,(key,item) in enumerate(value.items()):
        letter = get_column_letter(index+1)
        ws[letter+"1"] = "Animal " + key
        for index, length in enumerate(item):
            ws[letter+str(index+2)] = length
            
wb.save("Layer Five List.xlsx")

In [22]:
from scipy.stats import ttest_ind, ttest_rel

t, p = ttest_ind(samplelist["TM349"],samplelist["TM340"])

In [24]:
pvalue = "{0:.10f}".format(p)
tvalue = "{0:.10f}".format(t)

In [14]:

for data in Global.datalist:
        face_file_name = "AIS " + "{0:0=3}".format(data.index) + ".jpg"
        pixellength = data.aislength
        physicallength = data.aisphysicallength
        picturelength = data.piclength
        vectorlength = data.vectorlength
        ws.append([face_file_name, pixellength, physicallength,picturelength,vectorlength,picturelength/vectorlength])


# Save the file
filepath = os.path.join(Global.dirname, "AIS lengths.xlsx")
wb.save(filepath)

40

In [25]:
print("T-Wert: {0} P-Wert: {1}".format(tvalue,pvalue))

T-Wert: 4.8303691249 P-Wert: 0.0000021309
